<a href="https://colab.research.google.com/github/thingumajig/colab-experiments/blob/master/train_BERT_BiLSTM_CRF_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

## clone from git

In [1]:
#!pip install --upgrade tensorflow-gpu
#!pip install seqeval
!pip install bert-tensorflow

!pip list | grep tensorflow
#!ps ax | grep python


  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval
     |████████████████████████████████| 71kB 3.3MB/s 
bert-tensorflow          1.0.1                
mesh-tensorflow          0.0.5                
tensorflow               1.14.0               
tensorflow-estimator     1.14.0               
tensorflow-hub           0.5.0                
tensorflow-metadata      0.14.0               
tensorflow-probability   0.7.0                


In [2]:

import sys

# bert_path = 'bert'
# !test -d $bert_path || git clone https://github.com/google-research/bert $bert_path
# if not bert_path in sys.path:
#   sys.path.insert(0, bert_path)



bert_ner_path = 'bert-ner'  
!test -d $bert_ner_path || git clone https://github.com/thingumajig/colab-experiments $bert_ner_path

if not bert_ner_path in sys.path:
  sys.path.insert(0, bert_ner_path)

#!cp /content/BERT-NER/metrics.py /content/metrics.py



Cloning into 'bert-ner'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 69 (delta 33), reused 21 (delta 7), pack-reused 0
Unpacking objects: 100% (69/69), done.


In [3]:
#!rm -rfv $bert_ner_path/bert
!git -C $bert_ner_path pull 

Already up to date.


##Model

In [4]:
#@title Select model:
import os

bert_model = "https://storage.googleapis.com/bert_models/2019_05_30/wwm_cased_L-24_H-1024_A-16.zip" #@param ["https://storage.googleapis.com/bert_models/2019_05_30/wwm_cased_L-24_H-1024_A-16.zip", "https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip"]
bert_model_zip = os.path.basename(bert_model)
bert_model_dir = os.path.splitext(bert_model_zip)[0]

!test -d $bert_model_dir || wget $bert_model


!test -d $bert_model_dir || unzip /content/$bert_model_zip

--2019-07-26 12:16:24--  https://storage.googleapis.com/bert_models/2019_05_30/wwm_cased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.128, 2607:f8b0:4001:c0b::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1242589256 (1.2G) [application/zip]
Saving to: ‘wwm_cased_L-24_H-1024_A-16.zip’

wwm_cased_L-24_H-10 100%[===================>]   1.16G   109MB/s    in 10s     

2019-07-26 12:16:34 (117 MB/s) - ‘wwm_cased_L-24_H-1024_A-16.zip’ saved [1242589256/1242589256]

Archive:  /content/wwm_cased_L-24_H-1024_A-16.zip
   creating: wwm_cased_L-24_H-1024_A-16/
  inflating: wwm_cased_L-24_H-1024_A-16/bert_model.ckpt.meta  
  inflating: wwm_cased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001  
  inflating: wwm_cased_L-24_H-1024_A-16/vocab.txt  
  inflating: wwm_cased_L-24_H-1024_A-16/bert_model.ckpt.index  
  inflating: wwm_cased_L-24_H-

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


##init flags

In [11]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import os
import json
from bert import modeling
from bert import optimization
from bert import tokenization
import tensorflow as tf
import codecs
from tensorflow.contrib.layers.python.layers import initializers
from tensorflow.contrib import crf
from tensorflow.contrib import rnn, cudnn_rnn

from tensorflow.contrib.tpu import TPUEstimator, TPUConfig
from tensorflow.python.ops import math_ops

import tf_metrics
import pickle

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

flags = tf.flags

FLAGS = flags.FLAGS



# print(FLAGS.__dict__['__wrapped'])
def del_all_flags(FLAGS, r = []):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        if keys in r:
          FLAGS.__delattr__(keys)
 
# allows rerun cell
del_all_flags(FLAGS, r = ['bert_config_file','data_dir',
                         "task_name", "output_dir","init_checkpoint",
                         "do_lower_case", "do_train", "use_tpu", "do_predict",
                         "use_crf", "train_batch_size", "eval_batch_size",
                         "learning_rate", "num_train_epochs", "warmup_proportion",
                         "bert_dropout_rate", "bilstm_dropout_rate",
                          "save_checkpoints_steps", "save_summary_steps",
                         'data_config_path', "num_tpu_cores", "master",
                          "vocab_file", "keep_checkpoint_max",                          
                          'lstm_size', 'max_seq_length',
                          'predict_batch_size', 'iterations_per_loop',
                          'f',
                         ])
train_model_path = '/content/drive/My Drive/ner_checkpoints'
if not os.path.exists(train_model_path):
    os.makedirs(train_model_path)

flags.DEFINE_string("data_dir", f'/content/{bert_ner_path}/data/CoNLL', "The input datadir. ex) 'NERdata'")
flags.DEFINE_string("bert_config_file", os.path.join('/content',bert_model_dir,'bert_config.json'), "The config json file corresponding to the pre-trained BERT model. ex) 'bert_config.json'")

flags.DEFINE_string("task_name", 'NER', "The name of the task to train. ex) 'NER'")

flags.DEFINE_string("output_dir", train_model_path, "The output directory where the model checkpoints will be written. ex) 'output'")

flags.DEFINE_string("init_checkpoint", f"/content/{bert_model_dir}/bert_model.ckpt", "Initial checkpoint (usually from a pre-trained BERT model).")

flags.DEFINE_bool("do_lower_case", False, "Whether to lower case the input text.")

flags.DEFINE_integer("max_seq_length", 128, "The maximum total input sequence length after WordPiece tokenization.")

flags.DEFINE_bool("do_train", True, "Whether to run training.")
# flags.DEFINE_bool("do_predict", True, "Whether to run the model in inference mode on the test set.")

flags.DEFINE_bool("use_crf", True, "Whether to use CRF decoding.")

flags.DEFINE_integer("train_batch_size", 16, "Total batch size for training.")
flags.DEFINE_integer("eval_batch_size", 8, "Total batch size for eval.")
flags.DEFINE_integer("predict_batch_size", 8, "Total batch size for predict.")
flags.DEFINE_float("learning_rate", 5e-5, "The initial learning rate for Adam.")

flags.DEFINE_float("num_train_epochs", 5.0, "Total number of training epochs to perform.")
flags.DEFINE_float("warmup_proportion", 0.1, "Proportion of training to perform linear learning rate warmup for. E.g., 0.1 = 10% of training.")
flags.DEFINE_float("bert_dropout_rate", 0.2, "Proportion of dropout for bert embedding.")
flags.DEFINE_float("bilstm_dropout_rate", 0.2, "Proportion of dropout for bilstm.")
flags.DEFINE_integer("save_checkpoints_steps", 1000, "How often to save the model checkpoint.")
flags.DEFINE_integer("save_summary_steps", 100, "Save summaries every this many steps")
flags.DEFINE_integer("keep_checkpoint_max", 20, "The maximum number of recent checkpoint files to keep")
flags.DEFINE_integer("iterations_per_loop", 1000, "How many steps to make in each estimator call.")

flags.DEFINE_string("vocab_file", f'/content/{bert_model_dir}/vocab.txt', "The vocabulary file that the BERT model was trained on. ex) 'vocab.txt'")

flags.DEFINE_string("master", None, "[Optional] TensorFlow master URL.")

flags.DEFINE_bool("use_tpu", False, "Whether to use TPU or GPU/CPU.")
flags.DEFINE_integer( "num_tpu_cores", 8, "Only used if `use_tpu` is True. Total number of TPU cores to use.")

flags.DEFINE_string('data_config_path', 'data.conf', 'data config file, which save train and dev config')

flags.DEFINE_integer('lstm_size', 128, 'size of lstm units')

flags.DEFINE_string("f", 'f', "f") #????? error otherwise

print(FLAGS.__dict__['__wrapped'])



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:
  --bert_config_file: The config json file corresponding to the pre-trained BERT
    model. ex) 'bert_config.json'
    (default: '/content/wwm_cased_L-24_H-1024_A-16/bert_config.json')
  --bert_dropout_rate: Proportion of dropout for bert embedding.
    (default: '0.2')
    (a number)
  --bilstm_dropout_rate: Proportion of dropout for bilstm.
    (default: '0.2')
    (a number)
  --data_config_path: data config file, which save train and dev config
    (default: 'data.conf')
  --data_dir: The input datadir. ex) 'NERdata'
    (default: '/content/bert-ner/data/CoNLL')
  --[no]do_lower_case: Whether to lower case the input text.
    (default: 'false')
  --[no]do_train: Whether to run training.
    (default: 'true')
  --eval_batch_size: Total batch size for eval.
    (default: '8')
    (an integer)
  --f: f
    (default: 'f')
  --init_checkpoint: Initial checkpoint (usually from a pre-trained BERT model).
    (default: '/conten

## main

In [7]:
import bert_bilstm_model as m

processors = {
    "ner": m.NerProcessor
}

def create_estimator(label_list):
    bert_config = modeling.BertConfig.from_json_file(FLAGS.bert_config_file)
    if FLAGS.max_seq_length > bert_config.max_position_embeddings:
        raise ValueError(
            "Cannot use sequence length %d because the BERT model "
            "was only trained up to sequence length %d" %
            (FLAGS.max_seq_length, bert_config.max_position_embeddings))

    tokenizer = tokenization.FullTokenizer(
        vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)

    tpu_cluster_resolver = None
    if FLAGS.use_tpu and FLAGS.tpu_name:
        tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(
            FLAGS.tpu_name, zone=FLAGS.tpu_zone, project=FLAGS.gcp_project)
    is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2

    run_config = tf.contrib.tpu.RunConfig(
        cluster=tpu_cluster_resolver,
        master=FLAGS.master,
        model_dir=FLAGS.output_dir,
        save_checkpoints_steps=FLAGS.save_checkpoints_steps,
        keep_checkpoint_max=FLAGS.keep_checkpoint_max,
        save_summary_steps=FLAGS.save_summary_steps,
        tpu_config=tf.contrib.tpu.TPUConfig(
            iterations_per_loop=FLAGS.iterations_per_loop,
            num_shards=FLAGS.num_tpu_cores,
            per_host_input_for_training=is_per_host))

    model_fn = m.model_fn_builder(
        bert_config=bert_config,
        num_labels=len(label_list) + 1,  # 1 for '0' padding
        init_checkpoint=FLAGS.init_checkpoint,
        learning_rate=FLAGS.learning_rate,
        use_tpu=FLAGS.use_tpu,
        use_one_hot_embeddings=FLAGS.use_tpu)

    ws = None
    if os.path.exists(FLAGS.output_dir):
        print(f'================== use WarmStartSettings from {FLAGS.output_dir}')
        ws = tf.estimator.WarmStartSettings(ckpt_to_initialize_from=FLAGS.output_dir)
    estimator = tf.contrib.tpu.TPUEstimator(
        use_tpu=FLAGS.use_tpu,
        model_fn=model_fn,
        config=run_config,
        train_batch_size=FLAGS.train_batch_size,
        eval_batch_size=FLAGS.eval_batch_size,
        predict_batch_size=FLAGS.predict_batch_size,
        warm_start_from=ws
    )
    return estimator, tokenizer


def train(estimator, tokenizer, label_list):
  data_config_path = os.path.join(FLAGS.output_dir,FLAGS.data_config_path)  
  if os.path.exists(data_config_path):
      with codecs.open(data_config_path) as fd:
          print(f'=========== load existed config:{FLAGS.data_config_path}')
          data_config = json.load(fd)
  else:
      data_config = {}

  train_examples = None
  num_train_steps = None
  num_warmup_steps = None
  if len(data_config) == 0:
      train_examples = processor.get_train_examples(FLAGS.data_dir)
      num_train_steps = int((len(train_examples) / FLAGS.train_batch_size) * FLAGS.num_train_epochs)
      num_warmup_steps = int(num_train_steps * FLAGS.warmup_proportion)
      data_config['num_train_steps'] = num_train_steps
      data_config['num_warmup_steps'] = num_warmup_steps
      data_config['num_train_size'] = len(train_examples)
  else:
      num_train_steps = int(data_config['num_train_steps'])
      num_warmup_steps = int(data_config['num_warmup_steps'])

  # prepare train_input_fn
  if data_config.get('train.tf_record_path', '') == '':
      train_file = os.path.join(FLAGS.output_dir, "train.tf_record")
      m.filed_based_convert_examples_to_features(
          train_examples, label_list, FLAGS.max_seq_length, tokenizer, train_file)
  else:
      train_file = data_config.get('train.tf_record_path')
  print(f'================== train file: {train_file}')
  num_train_size = num_train_size = int(data_config['num_train_size'])
  print("***** Running training *****")
  print(f"  Num examples = {num_train_size} Batch size = {FLAGS.train_batch_size} Num steps = {num_train_steps}")
  train_input_fn = m.file_based_input_fn_builder(
      input_file=train_file,
      seq_length=FLAGS.max_seq_length,
      is_training=True,
      drop_remainder=True)
  # prepare eval_input_fn
  if data_config.get('eval.tf_record_path', '') == '':
      eval_examples = processor.get_dev_examples(FLAGS.data_dir)
      eval_file = os.path.join(FLAGS.output_dir, "eval.tf_record")
      m.filed_based_convert_examples_to_features(
          eval_examples, label_list, FLAGS.max_seq_length, tokenizer, eval_file)
      data_config['eval.tf_record_path'] = eval_file
      data_config['num_eval_size'] = len(eval_examples)
  else:
      eval_file = data_config['eval.tf_record_path']
  num_eval_size = data_config.get('num_eval_size', 0)
  print("***** Running evaluation *****")
  print(f"  Num examples = {num_eval_size} Batch size = {FLAGS.eval_batch_size}")
  eval_steps = None
  if FLAGS.use_tpu:
      eval_steps = int(num_eval_size / FLAGS.eval_batch_size)
  eval_drop_remainder = True if FLAGS.use_tpu else False
  eval_input_fn = m.file_based_input_fn_builder(
      input_file=eval_file,
      seq_length=FLAGS.max_seq_length,
      is_training=False,
      drop_remainder=eval_drop_remainder)
  # train and evaluate tf.estimator.experimental.
  hook = tf.estimator.experimental.stop_if_no_decrease_hook(
      estimator, 'eval_f', 3000, min_steps=30000, run_every_secs=120)
  train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=num_train_steps, hooks=[hook])
  eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn, throttle_secs=120)
  tp = tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
  result = tp[0]

  output_eval_file = os.path.join(FLAGS.output_dir, "eval_results.txt")
  with codecs.open(output_eval_file, "w", encoding='utf-8') as writer:
      print("***** Eval results *****")
      for key in sorted(result.keys()):
          print(f"  {key} = {str(result[key])}")
          writer.write(f"{key} = {str(result[key])}\n")


  if not os.path.exists(data_config_path):
      with codecs.open(data_config_path, 'a', encoding='utf-8') as fd:
          json.dump(data_config, fd)


def predict(estimator, tokenizer, label_list, text):

    # prepare predict_input_fn
    # predict_examples = processor.get_test_examples(FLAGS.data_dir)
    # predict_examples = processor.get_predict_examples(os.path.join(FLAGS.output_dir, filename))
    predict_examples = processor.get_predict_examples_from_str(text)
    print(f'================= Number of predict examples:{len(predict_examples)}')

    predict_file = os.path.join(FLAGS.output_dir, "predict.tf_record")
    m.filed_based_convert_examples_to_features(predict_examples, label_list,
                                               FLAGS.max_seq_length, tokenizer,
                                               predict_file, mode="test")

    print("***** Running prediction*****")
    print(f"  Num examples = {len(predict_examples)} Batch size = {FLAGS.predict_batch_size}")
    if FLAGS.use_tpu:
        # Warning: According to tpu_estimator.py Prediction on TPU is an
        # experimental feature and hence not supported here
        raise ValueError("Prediction in TPU not supported")
    predict_drop_remainder = True if FLAGS.use_tpu else False
    predict_input_fn = m.file_based_input_fn_builder(
        input_file=predict_file,
        seq_length=FLAGS.max_seq_length,
        is_training=False,
        drop_remainder=predict_drop_remainder)

    # predict
    result = estimator.predict(input_fn=predict_input_fn)
    
    return predict_examples, result 



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# run

In [12]:
tf.logging.set_verbosity(tf.logging.INFO)
task_name = FLAGS.task_name.lower()
if task_name not in processors:
    raise ValueError("Task not found: %s" % (task_name))
processor = processors[task_name]()

label_list = processor.get_labels()

estimator, tokenizer = create_estimator(label_list)
train(estimator, tokenizer, label_list)

W0726 12:43:53.873618 140641613690752 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7fe93fd22488>) includes params argument, but params are not passed to Estimator.
I0726 12:43:53.876022 140641613690752 estimator.py:209] Using config: {'_model_dir': '/content/drive/My Drive/ner_checkpoints', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 20, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe93fa16dd8>, '_task_type': 'worker', '_task_id':

================== use WarmStartSettings from /content/drive/My Drive/ner_checkpoints
Writing example 0 of 14041
*** Example ***
guid: train-0
tokens: EU rejects German call to boycott British la ##mb .
input_ids: 101 7270 22961 1528 1840 1106 21423 1418 2495 12913 119 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

I0726 12:44:05.681118 140641613690752 estimator_training.py:186] Not using Distribute Coordinator.
I0726 12:44:05.682183 140641613690752 training.py:612] Running training and evaluation locally (non-distributed).
I0726 12:44:05.684036 140641613690752 training.py:700] Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 1000 or save_checkpoints_secs None.
W0726 12:44:05.723655 140641613690752 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0726 12:44:05.759764 140641613690752 deprecation.py:323] From bert-ner/bert_bilstm_model.py:334: map_and

***** Running evaluation *****
  Num examples = 3250 Batch size = 8
input_fn: batch_size=16 is_training=True
*** Features ***
  name = input_ids, shape = (16, 128)
  name = input_mask, shape = (16, 128)
  name = label_ids, shape = (16, 128)
  name = segment_ids, shape = (16, 128)
shape of input_ids (16, 128)
shape of label_ids (16, 128)
estimator mode: train


W0726 12:44:05.888153 140641613690752 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/bert/modeling.py:671: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0726 12:44:11.060917 140641613690752 deprecation.py:323] From bert-ner/bert_bilstm_model.py:359: dropout (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dropout instead.
W0726 12:44:11.125846 140641613690752 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


...

####################
shape of output_layer: (?, 128, 1024)
embedding_size:1024
seq_length:128
shape of logit (?, 128, 11)
shape of loss ()
shape of per_example_loss (?,)
num labels:11
####################
shape of pred_ids (?, 128)


W0726 13:09:48.223182 140641613690752 deprecation_wrapper.py:119] From bert-ner/bert_bilstm_model.py:545: The name tf.metrics.accuracy is deprecated. Please use tf.compat.v1.metrics.accuracy instead.

W0726 13:09:48.284720 140641613690752 deprecation_wrapper.py:119] From bert-ner/bert_bilstm_model.py:546: The name tf.metrics.mean is deprecated. Please use tf.compat.v1.metrics.mean instead.

I0726 13:09:48.302470 140641613690752 estimator.py:1147] Done calling model_fn.
I0726 13:09:48.324428 140641613690752 evaluation.py:255] Starting evaluation at 2019-07-26T13:09:48Z
I0726 13:09:49.867372 140641613690752 monitored_session.py:240] Graph was finalized.
I0726 13:09:49.879425 140641613690752 saver.py:1280] Restoring parameters from /content/drive/My Drive/ner_checkpoints/model.ckpt-3632
I0726 13:09:58.212820 140641613690752 session_manager.py:500] Running local_init_op.
I0726 13:09:58.320111 140641613690752 session_manager.py:502] Done running local_init_op.
I0726 13:10:02.839045 14064161

input_fn: batch_size=8 is_training=False
*** Features ***
  name = input_ids, shape = (?, 128)
  name = input_mask, shape = (?, 128)
  name = label_ids, shape = (?, 128)
  name = segment_ids, shape = (?, 128)
shape of input_ids (?, 128)
shape of label_ids (?, 128)
estimator mode: eval
seq_length 128
lengths Tensor("Sum:0", shape=(?,), dtype=int32)
####################
shape of output_layer: (?, 128, 1024)
embedding_size:1024
seq_length:128
shape of logit (?, 128, 11)
shape of loss ()
shape of per_example_loss (?,)
num labels:11
####################
shape of pred_ids (?, 128)


I0726 13:28:32.374137 140641613690752 estimator.py:1147] Done calling model_fn.
I0726 13:28:32.400141 140641613690752 evaluation.py:255] Starting evaluation at 2019-07-26T13:28:32Z
I0726 13:28:33.335174 140641613690752 monitored_session.py:240] Graph was finalized.
I0726 13:28:33.373383 140641613690752 saver.py:1280] Restoring parameters from /content/drive/My Drive/ner_checkpoints/model.ckpt-4387
I0726 13:28:42.637365 140641613690752 session_manager.py:500] Running local_init_op.
I0726 13:28:42.753071 140641613690752 session_manager.py:502] Done running local_init_op.
I0726 13:28:47.151637 140641613690752 evaluation.py:167] Evaluation [10/100]
I0726 13:28:49.771575 140641613690752 evaluation.py:167] Evaluation [20/100]
I0726 13:28:52.418902 140641613690752 evaluation.py:167] Evaluation [30/100]
I0726 13:28:55.067413 140641613690752 evaluation.py:167] Evaluation [40/100]
I0726 13:28:57.772783 140641613690752 evaluation.py:167] Evaluation [50/100]
I0726 13:29:00.310799 140641613690752 e

***** Eval results *****
  eval_accuracy = 0.9862786
  eval_f = 0.95323485
  eval_loss = 1.4167155
  eval_precision = 0.9534658
  eval_recall = 0.9530039
  global_step = 4387
  loss = 1.4167155


In [0]:
#print(FLAGS.__dict__['__wrapped'])
flags.DEFINE_string("f", 'f', "f")
FLAGS.data_dir

In [13]:
!zip model.ckpt-4387.zip "/content/drive/My Drive/ner_checkpoints/model.ckpt-4387.data-00000-of-00001" "/content/drive/My Drive/ner_checkpoints/model.ckpt-4387.index" "/content/drive/My Drive/ner_checkpoints/model.ckpt-4387.meta" "/content/drive/My Drive/ner_checkpoints/label2id.pkl"

  adding: content/drive/My Drive/ner_checkpoints/model.ckpt-4387.data-00000-of-00001 (deflated 11%)
  adding: content/drive/My Drive/ner_checkpoints/model.ckpt-4387.index (deflated 71%)
  adding: content/drive/My Drive/ner_checkpoints/model.ckpt-4387.meta (deflated 93%)
  adding: content/drive/My Drive/ner_checkpoints/label2id.pkl (deflated 33%)


In [22]:
# %tensorboard — logdir /content/drive/My Drive/ner_checkpoints
!pip install tensorboardcolab
from tensorboardcolab import *
tbc=TensorBoardColab(graph_path='/content/drive/My Drive/ner_checkpoints')


Wait for 8 seconds...
TensorBoard link:
https://56d9591a.ngrok.io


In [23]:
#!kill -9 1688
!ps ax 

    PID TTY      STAT   TIME COMMAND
      1 ?        Ss     0:00 /bin/bash -e /datalab/run.sh
     11 ?        Sl     0:08 /tools/node/bin/node /datalab/web/app.js
     29 ?        Sl     0:20 /usr/bin/python2 /usr/local/bin/jupyter-notebook --
    128 ?        Ss     0:00 tail -n +0 -F /root/.config/Google/DriveFS/Logs/dri
    141 ?        Ssl   45:56 /usr/bin/python3 -m ipykernel_launcher -f /root/.lo
    248 ?        Sl     0:00 /opt/google/drive/drive --features=opendir_timeout_
    339 ?        Sl     2:44 /opt/google/drive/drive --crash_handler_token=dummy
    389 ?        S      0:00 tail -n +0 -F /root/.config/Google/DriveFS/Logs/dri
    390 ?        S      0:00 grep --color=auto --line-buffered QueryManager time
    391 ?        S      0:00 /usr/bin/python3 -Wignore:::pip._internal.cli.base_
   1798 ?        Sl     0:00 ngrok http 6006
   1828 ?        R      0:00 ps ax
